In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')


In [2]:
# -----------------------------------
# ロジスティック回帰用の特徴量の作成
# -----------------------------------

from sklearn.preprocessing import OneHotEncoder

# 元データをコピーする
train_x2 = train.drop(['Survived'], axis=1)
test_x2 = test.copy()

train_y = train['Survived'].copy()

# 変数PassengerIdを除外する
train_x2 = train_x2.drop(['PassengerId'], axis=1)
test_x2 = test_x2.drop(['PassengerId'], axis=1)

# 変数Name, Ticket, Cabinを除外する
train_x2 = train_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x2 = test_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)


# one-hot encodingを行う
cat_cols = ['Sex', 'Embarked', 'Pclass']
ohe = OneHotEncoder(categories='auto', sparse=False)
ohe.fit(train_x2[cat_cols].fillna('NA'))

# one-hot encodingのダミー変数の列名を作成する
ohe_columns = []
for i, c in enumerate(cat_cols):
    ohe_columns += [f'{c}_{v}' for v in ohe.categories_[i]]

# one-hot encodingによる変換を行う
ohe_train_x2 = pd.DataFrame(ohe.transform(train_x2[cat_cols].fillna('NA')), columns=ohe_columns)
ohe_test_x2 = pd.DataFrame(ohe.transform(test_x2[cat_cols].fillna('NA')), columns=ohe_columns)

# one-hot encoding済みの変数を除外する
train_x2 = train_x2.drop(cat_cols, axis=1)
test_x2 = test_x2.drop(cat_cols, axis=1)

# one-hot encodingで変換された変数を結合する
train_x2 = pd.concat([train_x2, ohe_train_x2], axis=1)
test_x2 = pd.concat([test_x2, ohe_test_x2], axis=1)

# 数値変数の欠損値を学習データの平均で埋める
num_cols = ['Age', 'SibSp', 'Parch', 'Fare']
for col in num_cols:
    train_x2[col].fillna(train_x2[col].mean(), inplace=True)
    test_x2[col].fillna(train_x2[col].mean(), inplace=True)

# 変数Fareを対数変換する
train_x2['Fare'] = np.log1p(train_x2['Fare'])
test_x2['Fare'] = np.log1p(test_x2['Fare'])

#train_x2データ

In [3]:
# -----------------------------------
# モデルチューニング
# -----------------------------------
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

import itertools

# チューニング候補とするパラメータを準備する
param_space = {
    'max_depth': [3,4,5,6,7],
    'min_child_weight': [1,2,3,4,5]
}

# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space['max_depth'], param_space['min_child_weight'])

# 各パラメータの組み合わせ、それに対するスコアを保存するリスト
params = []
scores = []

# 各パラメータの組み合わせごとに、クロスバリデーションで評価を行う
for max_depth, min_child_weight in param_combinations:

    score_folds = []
    # クロスバリデーションを行う
    # 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
    kf = KFold(n_splits=10, shuffle=True, random_state=123456)
    
    for tr_idx, va_idx in kf.split(train_x2):
        # 学習データを学習データとバリデーションデータに分ける
        tr_x, va_x = train_x2.iloc[tr_idx], train_x2.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

        # モデルの学習を行う
        model = XGBClassifier(n_estimators=10, random_state=71,
                              max_depth=max_depth, min_child_weight=min_child_weight)
        model.fit(tr_x, tr_y)

        # バリデーションデータでのスコアを計算し、保存する
        va_pred = model.predict_proba(va_x)[:, 1]
        logloss = log_loss(va_y, va_pred)
        score_folds.append(logloss)
  
    # 各foldのスコアを平均する
    score_mean = np.mean(score_folds)

    # パラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)
    
# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')

#モデルの学習とparameterの最適化を同時に行われる（２重のfor文）
#各parameterの比較の間もモデルの学習は続く

max_depth: 6, min_child_weight: 5


In [4]:
scores
#最小値が最も良い

[0.42080325873836066,
 0.42106671122520156,
 0.4238107197442454,
 0.4225077834574694,
 0.42275288123363036,
 0.42279843135407835,
 0.42218378673355267,
 0.419502918200193,
 0.4216398805933313,
 0.42500555549342456,
 0.4278397466389744,
 0.42507434628736435,
 0.4219275348092174,
 0.42061180808594195,
 0.4228948280392924,
 0.43322715852976995,
 0.42957339358459057,
 0.4237644432626488,
 0.4227260139402695,
 0.4174966352982691,
 0.43436869576312925,
 0.43233899256491226,
 0.4214685529059313,
 0.42244982808865555,
 0.421168736646489]

In [5]:
params
#組み合わせは25通り

[(3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5)]

In [6]:
# -----------------------------------
# predict
# -----------------------------------

from sklearn.linear_model import LogisticRegression

# xgboostモデル
model_xgb = XGBClassifier(n_estimators=20, random_state=71, max_depth=best_param[0], min_child_weight=best_param[1])
model_xgb.fit(train_x2, train_y)
pred_xgb = model_xgb.predict_proba(test_x2)[:, 1]


In [7]:
# -----------------------------------
# アンサンブル
# -----------------------------------

# 予測値の加重平均をとる
pred3 = pred_xgb
pred_label3 = np.where(pred3 > 0.5, 1, 0)

# 提出用ファイルの作成
submission3 = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': pred_label3})
submission3.to_csv('submission_third.csv', index=False)
